In [2]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
url_initial = "https://pt.wikisource.org/wiki/Categoria:Literatura_brasileira"

In [4]:
response = re.get(url_initial)
soup = bs(response.text, 'html5lib')
links = soup.find('div', class_='mw-category').find_all('a')

In [5]:
def f_links(url):
    response = re.get(url)
    soup = bs(response.text, 'html5lib')
    #links = soup.find('div', class_='content').find_all('a')
    links = soup.find_all('a')
    return links

In [7]:
def buscar(chave, profundidade=1):
    dic = {'profundidade_%s'%i: [] for i in range(profundidade+1)}
    dic['profundidade_0'] = links
    for key in range(1, profundidade+1):
        for a in dic['profundidade_%s'%(key-1)]:
            try:
                if a['href'][:5] == '/wiki':
                    link = 'https://pt.m.wikisource.org' + a['href']
                elif 'https://pt' in a['href']:
                    link = a['href']
                dic['profundidade_%s'%(key)].extend(f_links(link))
            except KeyError:
                pass
    return dic

In [ ]:
d = buscar('qualquer coisa', 2)

In [ ]:
d